In [87]:
from LambertSolarSystem import LambertSolarSystem
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from PyKEP import AU, DAY2SEC, DAY2YEAR, MU_SUN, epoch
from PyKEP.orbit_plots import plot_planet, plot_lambert
import PyKEP as pkp
import numpy as np
from scipy.optimize import brent, newton


system = LambertSolarSystem(87464)

posFunc = system.getPositionsFunction()
velFunc = system.getVelocityFunction()

 # finds the optimal transfer between the planets
 Takes a start value t and two planets


In [5]:
# ----------------------- First init ---------------------------------------------------
SEC2YEAR = 1./DAY2YEAR*DAY2SEC
t = 14.32
planetsPos0 = posFunc(t)[:]
planetsVel0 = velFunc(t)[:]

A = 0; B = 4; C = 5
planets = np.array((A,B,C))
# Add 3. dimension
pos0 = np.r_[planetsPos0, [np.zeros(7)]]
vel0 = np.r_[planetsVel0, [np.zeros(7)]]
rA = pos0[:,A] * AU
vA = vel0[:,A] * AU / SEC2YEAR
rB = pos0[:,B] * AU
vB = vel0[:,B] * AU / SEC2YEAR
rC = pos0[:,C] * AU
vC = vel0[:,C] * AU / SEC2YEAR

mu_star = system.starMass * MU_SUN
mu_planet = system.mass[planets]  * MU_SUN
radius = system.radius[planets] * 1000
safe_radius = radius *1.2

planetA = pkp.planet.keplerian(epoch(0), rA, vA, mu_star, mu_planet[0], radius[0], safe_radius[0], 'A')
planetB = pkp.planet.keplerian(epoch(0), rB, vB, mu_star, mu_planet[1], radius[1], safe_radius[1], 'B')
planetC = pkp.planet.keplerian(epoch(0), rC, vC, mu_star, mu_planet[2], radius[2], safe_radius[2], 'C')

In [102]:
res = 0.2
times0 = np.arange(0,1,res)
times1 = np.arange(3,4,res)
times2 = np.arange(4,5,res)

def f(r_p, ain, aout, angle):
    return np.arcsin(ain/(ain+r_p)) + np.arcsin(ain/ain+r_p) - angle

def f2(r_p, v2in, v2out, angle):
    #print r_p
    a = np.arcsin(1/(1+v2in*r_p)) 
    b = np.arcsin(1/(1+v2out*r_p)) 
    #print a, b, angle
    #print '---------------'
  
    return a + b - angle

def angle(vin, vout):
    return np.arccos(np.dot(vin,vout)/(np.linalg.norm(vin)*np.linalg.norm(vout)))

for T0 in times0:
    for T1 in times1:
        for T2 in times2:

            t0 = epoch(T0/DAY2YEAR)
            t1 = epoch((T0+T1)/DAY2YEAR)
            t2 = epoch((T0+T1+T2)/DAY2YEAR)
            dt1 = (t1.mjd2000 - t0.mjd2000)*DAY2SEC
            dt2 = (t2.mjd2000 - t1.mjd2000)*DAY2SEC
            
            r1, v1 = planetA.eph(t0)
            r2, v2 = planetB.eph(t1)
            r3, v3 = planetC.eph(t2)
    
            l1    = pkp.lambert_problem(r1,r2,dt1,mu_star)
            vA    = np.array(l1.get_v1()[0]) 
            vBin  = np.array(l1.get_v2()[0])
            
            l2    = pkp.lambert_problem(r2,r3,dt2,mu_star)
            vBout = np.array(l2.get_v1()[0])
            vC    = np.array(l2.get_v2()[0])
            
     
            vBin_r  = vBin - v2
            vBout_r = vBout - v2
            #ain = 1/np.dot(vBin_r, vBin_r)
            #aout = 1/np.dot(vBout_r, vBout_r)
            vr2in  = np.dot(vBin_r, vBin_r) #Velocity relative to planet squared
            vr2out = np.dot(vBout_r, vBout_r)
            ang = angle(vBin_r, vBout_r)

            print ' ======================'
            rp = brent(f2,  args = (vr2in,vr2out,ang))    
            
            #rp = brent(f, args = (ain,aout,ang))   
            print rp
            print f(rp, vr2in, vr2out, ang)
            

            
            
               

103677.781129
nan
64076.774136
nan
167748.320809
nan
64076.774136
nan
103677.781129
nan
64077.3916877
nan
167748.320809
nan
271441.431711
nan
103673.545061
nan
271441.431797
nan
167748.320809
nan
64077.3919623
nan
103673.545061
nan
103679.399162
nan
167748.320809
nan
103677.781129
nan
64076.774136
nan
167759.411016
nan
103677.781129
nan
167748.320809
nan
64077.3916625
nan
103677.781129
nan
64075.7741361
nan
39601.2420641
nan
103673.545061
nan
103673.545061
nan
64076.774136
nan
103679.399162
nan
167755.174911
nan
39601.3876765
nan
103673.545061
nan
167748.320809
nan
167755.174911
nan
103680.398375
nan
167776.146434
nan
64075.7741361
nan
271441.431487
nan
167782.641336
nan
39601.332234
nan
64076.774136
nan
64076.774136
nan
64075.7741361
nan
167748.320809
nan
64075.7741361
nan
103677.781129
nan
167755.174911
nan
103677.781129
nan
103680.017196
nan
271434.576681
nan
103679.399162
nan
103677.781129
nan
39601.0059962
nan
103677.781129
nan
167755.174911
nan
103677.781129
nan
167759.411809
nan

In [2]:
       
            
res = 10000
time_array = np.linspace(0.2,4,res)
time_use_arr = np.zeros_like(time_array)
v_arr1 = np.zeros_like(time_array)
vBin_arr = np.zeros((res,3))
# ----------------------Find best GA-times ----------------------

for j,time_use in enumerate(time_array):
    t1 = epoch(0)            #relative to t0
    t2 = epoch(time_use/DAY2YEAR)   #relative to t0
    dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC


    r1, v1 = planetA.eph(t1)
    r2, v2 = planetB.eph(t2)
    
    l = pkp.lambert_problem(r1,r2,dt,mu_star)

    vA = l.get_v1()
    vB = np.array(l.get_v2())
    vel =  np.linalg.norm(vA) 
    time_use_arr[j] = time_use
    v_arr1[j] =  vel
    vBin_arr[j] = vB


best_v = np.amin(v_arr1)
print 'min v' ,best_v

i = np.argmin(v_arr1)
vBin = vBin_arr[i]

optimal_time1 = time_use_arr[i]
print 'optimal time use:', optimal_time1


time_array = np.linspace(2,8,res)
time_use_arr = np.zeros_like(time_array)
v_arr2 = np.zeros_like(time_array)
vBout_arr = np.zeros((res,3))

for j,time_use in enumerate(time_array):
    t1 = epoch(optimal_time1)            #relative to t0
    t2 = epoch((optimal_time1+time_use)/DAY2YEAR)   #relative to t0
    dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC


    r1, v1 = planetB.eph(t1)
    r2, v2 = planetC.eph(t2)
    
    l = pkp.lambert_problem(r1,r2,dt,mu_star)
    
    vB = np.array(l.get_v1()[0])
    #vC = l.get_v2()
    vel =  np.linalg.norm(vB-vBin)
    #print vB

    vBout_arr[j] = vB
    
    time_use_arr[j] = time_use
    v_arr2[j] =  vel

best_v = np.amin(v_arr2)
print 'min v' ,best_v

i = np.argmin(v_arr2)

vBout = vBout_arr[i]
print vBin, vBout
optimal_time2 = time_use_arr[i]
print 'optimal time use:', optimal_time2

min v 24884.1395345
optimal time use: 2.70141014101
min v 8434.79668366
[-14355.30165608   4171.47326116      0.        ] [-12571.94094201  12415.58750456      0.        ]
optimal time use: 5.36933693369


In [3]:
fig = plt.figure()
ax = fig.gca(projection = '3d')

t1 = epoch(0)            #relative to t0
t2 = epoch(optimal_time1/DAY2YEAR)   #relative to t0
dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC

r1, v1 = planetA.eph(t1)
r2, v2 = planetB.eph(t2)
l = pkp.lambert_problem(r1,r2,dt,mu_star)
norm = np.linalg.norm
print norm(l.get_v1())
print norm(v1)

plot_planet(planetA, t0 = epoch(0), legend = True, units = AU, ax = ax)
plot_planet(planetB, t0 = epoch(optimal_time1/DAY2YEAR), legend = True, units = AU, ax = ax)

for sol in range(l.get_Nmax()+1):
    plot_lambert(l, sol = sol, legend=True, units =AU, ax=ax)

t1 = epoch(optimal_time1/DAY2YEAR)            #relative to t0
t2 = epoch((optimal_time1+optimal_time2)/DAY2YEAR)   #relative to t0
dt = (t2.mjd2000 - t1.mjd2000)*DAY2SEC

r1, v1 = planetB.eph(t1)
r2, v2 = planetC.eph(t2)
l = pkp.lambert_problem(r1,r2,dt,mu_star)
norm = np.linalg.norm
print norm(l.get_v1())
print norm(v1)

plot_planet(planetB, t0 = t1, legend = True, units = AU, ax = ax)
plot_planet(planetC, t0 = t2, legend = True, units = AU, ax = ax)

for sol in range(l.get_Nmax()+1):
    plot_lambert(l, sol = sol, legend=True, units =AU, ax=ax)

plt.show()

24884.1395345
22277.3503102
17988.1754917
17259.4009884


#### Have found a way to get best lambert from one planet to another. Now lets find the best from one to another via a third!
